In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv


mmCoursesMeta = {
    'buslead': {
        "buslead1x": {"title": "UQx_BUSLEAD1x",  'grade': 0.65},
        "buslead2x": {"title": "UQx_BUSLEAD2x",  'grade': 0.65},
        "buslead3x": {"title": "UQx_BUSLEAD3x",  'grade': 0.65},
        "buslead4x": {"title": "UQx_BUSLEAD4x",  'grade': 0.65},
        "buslead5x": {"title": "UQx_BUSLEAD5x",  'grade': 0.65},
    },
    'corpinn': {
        "corpinn1x": {"title": "UQx_CORPINN1x", 'grade': 0.7},
        "corpinn2x": {"title": "UQx_CORPINN2x",  'grade': 0.7},
        "corpinn3x": {"title": "UQx_CORPINN3x",  'grade': 0.7},
        "corpinn4x": {"title": "UQx_CORPINN4x",  'grade': 0.7},
        "corpinn5x": {"title": "UQx_CORPINN5x",  'grade': 0.5},
    },
    'engy': {
        "engy0x": {"title": "UQx_ENGY0x", 'grade': 0.7},
        "engy1x": {"title": "UQx_ENGY1x", 'grade': 0.7},
        "engy2x": {"title": "UQx_ENGY2x", 'grade': 0.7},
        "engy3x": {"title": "UQx_ENGY3x", 'grade': 0.7},
        "engycapx": {"title": "UQx_ENGYCAPx", 'grade': 0.7},
    },
    'lgdm': {
        "lgdm1x": {"title": "UQx_LGDM1x", 'grade': 0.65},
        "lgdm2x": {"title": "UQx_LGDM2x",  'grade': 0.65},
        "lgdm3x": {"title": "UQx_LGDM3x",  'grade': 0.65},
        "lgdm4x": {"title": "UQx_LGDM4x",  'grade': 0.65},
        "lgdm5x": {"title": "UQx_LGDM5x",  'grade': 0.65},
    },
}
mmCoursesSeries = {
    'buslead': ['buslead1x', 'buslead2x', 'buslead3x', 'buslead4x'],
    'corpinn': ['corpinn1x', 'corpinn2x', 'corpinn3x', 'corpinn4x'],
    'engy': ['engy0x', 'engy1x', 'engy2x', 'engy3x'],
    'lgdm': ['lgdm1x', 'lgdm2x', 'lgdm3x', 'lgdm4x'],
}


def update_course_instancedetails_auditpassed():
    print('update_course_instancedetails_auditpassed')
    ci_query = 'SELECT * FROM course_instancedetails'
    result = sqlite_connection.execute(ci_query)
    for row in result:
        update_id = int(row[0])
        course = row[6]
        pass_line = micromasters_courses[course]['grade']
        #print('pass_line', pass_line)
        grade = row[2]
        enrollment_track = row[3]
        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('N', update_id)
        if (enrollment_track == 'audit') and (grade >= pass_line):
            update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('Y', update_id)
        sqlite_connection.execute(update_ci)
        
        
def micromasters_courses_add_last_run():
    for course in micromasters_courses:
        query = "SELECT course_instance FROM course_instancedetails WHERE course='%s' ORDER BY year DESC, term DESC;" % (course)
        query_result = sqlite_connection.execute(query).fetchone()
        if query_result is not None:
            micromasters_courses[course]['last_run'] = query_result[0]
    

def find_auditpassed(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']
    
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.audit_passed = 'Y'" % (trigger_run)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 0
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)
    
def find_certificate_eligible(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']

    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.certificate_eligible = 'Y'" % (trigger_run)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 1
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)


def find_auditpassed_next_course(course, record):
    #print('find_next_course', course, record)
    not_certificated = []
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    if course in not_certificated:
        not_certificated.remove(course)
    return not_certificated[0]

def find_certificate_eligible_next_course(record):
    not_certificated = []
    
    #print('record', record)
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    return not_certificated[0]
    
    
pointed_course_instances = {
    'lgdm1x': {
        'current_run': {'title': '', 'link': '', 'instance_code': 'UQx_LGDM1x_1T2017'},
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},
    },
    'lgdm2x': {
        'current_run':{'title': '', 'link': '', 'instance_code': 'UQx_LGDM2x_2T2017'},
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},
    },
    'lgdm3x': {
        'current_run':{'title': '', 'link': '', 'instance_code': 'UQx_LGDM3x_1T2018'}, 
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},       
    },
    'lgdm4x': {
        'current_run':{'title': '', 'link': '', 'instance_code': 'UQx_LGDM4x_1T2018'},
        'coming_run': {'title': '', 'link': '', 'instance_code': ''},        
    },
}  

filtered_students = []

mmCourse = 'lgdm'
course = 'lgdm2x'
table_name = mmCourse + '_coursecompletion'  
micromasters_courses = mmCoursesMeta[mmCourse]
micromasters_course_series = mmCoursesSeries[mmCourse]


sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()
  
    
# Prepare 1, update the column 'audit_passed' in the table {course_instancedetails}        
update_course_instancedetails_auditpassed()   

# Prepare 2, get the last run. Maybe not needed now, as we have the variable 'pointed_course_instances'
# micromasters_courses_add_last_run()

#current_course_code = pointed_course_instances[course]['current_run']['instance_code']
find_auditpassed(course)
find_certificate_eligible(course)

for item in filtered_students:
    query = "SELECT * FROM %s WHERE email = '%s'" % (table_name, item['email'])
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        item['no_certificate_eligible'] = record['no_certificate_eligible']
        if (item['trigger_course_verified'] == 0) and (item['no_certificate_eligible'] < 3):
            next_course = find_auditpassed_next_course(course, record)
            item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
            item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
        if (item['trigger_course_verified'] == 1) and (item['no_certificate_eligible'] < 4):
            next_course = find_certificate_eligible_next_course(record)
            item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
            item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
            
print(filtered_students)
    

print('The End.')

